In [4]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
import joblib
import os


MODEL_PATH = "models/knn_model.pkl"
MAPPERS_PATH = "models/mappers.pkl"

# User-Item Matrix
def create_item_user_matrix(df):
    """
    Cria uma matriz esparsa filme x usuário para recomendação item-item.
    """
    M = len(df['movieId'].unique())  # número de filmes
    N = len(df['userId'].unique())   # número de usuários
    
    # Mapear IDs para índices
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    
    # Mapear índices de volta para IDs
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    
    movie_index = [movie_mapper[i] for i in df['movieId']]
    user_index = [user_mapper[i] for i in df['userId']]

    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    
    return X, movie_mapper, user_mapper, movie_inv_mapper, user_inv_mapper

In [5]:
# Train and Save Model
def train_and_save_item_knn(X, movie_mapper, movie_inv_mapper, metric="cosine"):
    """
    Treina KNN item-item e salva o modelo + mapeamentos.
    """
    os.makedirs("models", exist_ok=True)

    knn = NearestNeighbors(algorithm="brute", metric=metric)
    knn.fit(X)

    joblib.dump(knn, MODEL_PATH)
    print(f"✅ Modelo salvo em: {os.path.abspath(MODEL_PATH)}")

    mappers = {
        "movie_mapper": movie_mapper,
        "movie_inv_mapper": movie_inv_mapper
    }
    joblib.dump(mappers, MAPPERS_PATH)
    print(f"✅ Mapeamentos salvos em: {os.path.abspath(MAPPERS_PATH)}")

In [6]:
import pandas as pd

# 1️⃣ Ler dados
ratings = pd.read_csv("../data/ratings.csv")

# 2️⃣ Criar matriz item-user (filme x usuário)
X, movie_mapper, user_mapper, movie_inv_mapper, user_inv_mapper = create_item_user_matrix(ratings)

# 3️⃣ Treinar e salvar modelo item-item
train_and_save_item_knn(X, movie_mapper, movie_inv_mapper)

✅ Modelo salvo em: d:\BACKUP\PROJETOS\MV-Recommender-System\recommender\models\knn_model.pkl
✅ Mapeamentos salvos em: d:\BACKUP\PROJETOS\MV-Recommender-System\recommender\models\mappers.pkl
